In [1]:
# import packages
import h5py
import numpy as np
from functions.preprocess import input_shaping, split_index
from functions.decoders import lstm_decoder
from functions.metrics import compute_rmse, compute_pearson
import time as timer
from tensorflow import random

In [2]:
seed = 2020 # random seed for reproducibility

print ("Starting simulation")
run_start = timer.time()

feature_list = ['sua_rate','mua_rate']
feature = feature_list[1] # select which spike feature: SUA=0, MUA=1

# specify filename to be processed (choose from the list available at https://zenodo.org/record/583331)
file_name = 'indy_20160915_01'          # file name
kinematic_folder = 'kinematic_data/'    # kinematic data folder
feature_folder = 'spike_data/features/' # spike features folder
result_folder = 'results/'              # results folder

Starting simulation


In [3]:
wdw_time = 0.256 # window size in second
lag = -32 # lag between kinematic and feature data (minus indicate feature lagging behaind kinematic)
delta_time = 0.004 # sampling interval in second
wdw_samp = int(round(wdw_time/delta_time))
ol_samp = wdw_samp-1

## Importing data to be processed

In [4]:
# open spike features from hdf5 file
feature_file = feature_folder+file_name+'_spike_features_'+str(int(wdw_time*1e3))+'ms.h5'
print ("Loading input features from file: "+feature_file)
with h5py.File(feature_file,'r') as f:
    input_feature = f[feature].value

Loading input features from file: spike_data/features/indy_20160915_01_spike_features_256ms.h5


c:\users\alexa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """


In [9]:
# open kinematic data from hdf5 file
kinematic_file = kinematic_folder+file_name+'_kinematic_data.h5'
print ("Loading kinematic data from file: "+kinematic_file)
#fields: ['cursor_acc', 'cursor_pos', 'cursor_time', 'cursor_vel', 'target_pos']
with h5py.File(kinematic_file,'r') as f:
    cursor_vel = f['cursor_vel'].value # in mm/s

Loading kinematic data from file: kinematic_data/indy_20160915_01_kinematic_data.h5


c:\users\alexa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  


## Preprocessing the data to fit a model

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
mm = MinMaxScaler()
ss = StandardScaler()


X_ss = ss.fit_transform(input_feature)
y_mm = mm.fit_transform(cursor_vel)

In [11]:
print(X_ss.shape, y_mm.shape)

(95262, 88) (95262, 2)


In [12]:
#first 80,000 for training

X_train = X_ss[:80000, :]
X_test = X_ss[80000:, :]

y_train = y_mm[:80000, :]
y_test = y_mm[80000:, :] 

In [13]:
print("Training Shape", X_train.shape, y_train.shape)
print("Testing Shape", X_test.shape, y_test.shape) 

Training Shape (80000, 88) (80000, 2)
Testing Shape (15262, 88) (15262, 2)


## Adjusting the data for pytorch model

In [14]:
import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable

In [20]:
#converting nparrays to tensors for pytorch

X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))


In [18]:
#reshaping to rows, timestamps, features

X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1]))

print("Training Shape", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape)

Training Shape torch.Size([80000, 1, 88]) torch.Size([80000, 2])
Testing Shape torch.Size([15262, 1, 88]) torch.Size([15262, 2])


## Building the LSTM model

In [21]:
class LSTM1(nn.Module):
  def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
    super(LSTM1, self).__init__()
    self.num_classes = num_classes #number of classes
    self.num_layers = num_layers #number of layers
    self.input_size = input_size #input size
    self.hidden_size = hidden_size #hidden state
    self.seq_length = seq_length #sequence length
 
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                      num_layers=num_layers, batch_first=True) #lstm
    self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
    self.fc = nn.Linear(128, num_classes) #fully connected last layer

    self.relu = nn.ReLU() 
    
  def forward(self,x):
    h_0 = Variable(torch.zeros(
          self.num_layers, x.size(0), self.hidden_size)) #hidden state
     
    c_0 = Variable(torch.zeros(
        self.num_layers, x.size(0), self.hidden_size)) #internal state
   
    # Propagate input through LSTM

    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
   
    hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next

    out = self.relu(hn)

    out = self.fc_1(out) #Fully connected layer

    out = self.relu(out) #relu

    out = self.fc(out) #Final Output
   
    return out 

In [30]:
num_epochs = 100 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 88 #number of features
hidden_size = 32 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 2 #number of output classes

In [31]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]) #our lstm class 

In [32]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate) 

## Running the LSTM on the data

In [33]:
for epoch in range(num_epochs):
  outputs = lstm1.forward(X_train_tensors_final) #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors)
 
  loss.backward() #calculates the loss of the loss function
 
  optimizer.step() #improve from loss, i.e backprop
  if epoch % 10 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.26248
Epoch: 10, loss: 0.13260
Epoch: 20, loss: 0.03459
Epoch: 30, loss: 0.02424
Epoch: 40, loss: 0.01293
Epoch: 50, loss: 0.00917
Epoch: 60, loss: 0.00753
Epoch: 70, loss: 0.00649
Epoch: 80, loss: 0.00583
Epoch: 90, loss: 0.00531


In [34]:
print(criterion(lstm1.forward(X_test_tensors_final), y_test_tensors))

tensor(0.0055, grad_fn=<MseLossBackward>)
